In [3]:
import qsharp
import matplotlib.pyplot as plt
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

[InstallKernelSpec] Installed kernelspec iqsharp in C:\ProgramData\jupyter\kernels\iqsharp


In [6]:
X_train=open('C:/Users\ceqcx\Downloads/0val_train')
X_train=X_train.readlines()
print(X_train)
X_train_new=[]
for i in X_train:
    if i != '\n':
        X_train_new.append(i)
for i in range(len(X_train_new)):
    X_train_new[i]=X_train_new[i].replace('\n',"")
    X_train_new[i]=X_train_new[i].split(',')
for i in range(len(X_train_new)):
    for j in range(len(X_train_new[i])):
            X_train_new[i][j]=float(X_train_new[i][j])
print(X_train_new)
X_train=X_train_new
X_test=open('C:/Users\ceqcx\Downloads/0val_test')
X_test=X_test.readlines()
print(X_test)
X_test_new=[]
for i in X_test:
    if i != '\n':
        X_test_new.append(i)
for i in range(len(X_test_new)):
    X_test_new[i]=X_test_new[i].replace('\n',"")
    X_test_new[i]=X_test_new[i].split(',')
for i in range(len(X_test_new)):
    for j in range(len(X_test_new[i])):
            X_test_new[i][j]=float(X_test_new[i][j])
print(X_test_new)
X_test=X_test_new
Y_train=open('C:/Users\ceqcx\Downloads/0check_train')
Y_train=Y_train.readlines()
Y_train=Y_train[0].split(',')
for i in range(len(Y_train)):
     Y_train[i]=int(Y_train[i])
print(len(Y_train))

Y_test=open('C:/Users\ceqcx\Downloads/0check_test')
Y_test=Y_test.readlines()
Y_test=Y_test[0].split(',')
for i in range(len(Y_test)):
     Y_test[i]=int(Y_test[i])
print(len(Y_test))

['0.008365672855715385,0.00949904478879219,0.009392920703974688,0.00943009430094301\n', '\n', '0.009202240141286924,0.009392910210146467,0.009195175004943643,0.00956676233429001\n', '\n', '0.008133293054167734,0.00949904478879219,0.009392920703974688,0.00943009430094301\n', '\n', '0.00801710315339391,0.009605179367437914,0.009244611429701404,0.007926745934126008\n', '\n', '0.009062812260358334,0.009286775631500745,0.009195175004943643,0.00970343036763701\n', '\n', '0.00848186275648921,0.008809170027594992,0.008947992881154835,0.007790077900779008\n', '\n', '0.008295958915251091,0.008968371895563576,0.00914573858018588,0.008610086100861008\n', '\n', '0.008737480538191624,0.009021439184886437,0.00914573858018588,0.00984009840098401\n', '\n', '0.008458624776334444,0.0090745064742093,0.009096302155428119,0.007790077900779008\n', '\n', '0.008249482954941559,0.008596900870303546,0.009640102827763496,0.009156758234249008\n', '\n', '0.00834243487556062,0.008809170027594992,0.009392920703974688

In [9]:
scaler=StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [10]:
%%qsharp

open Microsoft.Quantum.Math;
open Microsoft.Quantum.Arrays;

operation ZZFeatureMap(qs : Qubit[], data : Double[]) : Unit {
    let numberOfQubits = Length(qs);
    if (numberOfQubits != Length(data)) {
        fail "Data length must be equal to the number of qubits.";
    }
    for i in 0..numberOfQubits-1 {
        // Apply Z rotation to each qubit
        Rz(2.0 * PI() * data[i], qs[i]);
    }
    for i in 0..numberOfQubits-1 {
        for j in i+1..numberOfQubits-1 {
            // Apply ZZ interaction between each pair of qubits
            let theta = 2.0 * PI() * data[i] * data[j];
            CNOT(qs[i], qs[j]);
            Rz(theta, qs[j]);
            CNOT(qs[i], qs[j]);
        }
    }
}


operation RealAmplitudes(qubits : Qubit[], parameters : Double[]) : Unit {
    // Apply layer of rotation gates
    for idx in IndexRange(qubits) {
        Ry(parameters[idx], qubits[idx]);
    }

    // Apply a layer of CNOT gates
    for idx in 0 .. Length(qubits)-2 {
        for i in idx+1 .. Length(qubits)-1{
            CNOT(qubits[idx], qubits[i]);
        }
    }


    // Apply another layer of rotation gates
    for idx in IndexRange(qubits) {
        Ry(parameters[idx + Length(qubits)], qubits[idx]);
    }
}

operation quantumVariationalClassifier (qubits: Qubit[], x : Double[], θ : Double[]) : Unit {
    ZZFeatureMap(qubits, x);
    RealAmplitudes(qubits, θ);
}



In [13]:
def cost(weights, data, labels):
    # Make predictions using the quantum neural network, labels are the expected/real classification for the data
    predictions = quantumVariationalClassifier([], data, weights)
    mse = qml.mean_squared_error(labels, predictions)
    return mse

optimizer = qml.AdamOptimizer(stepsize=0.01)

steps = 100

#probably need to make this into a list or array to match theta type in other functions
weights = np.random.normal(0, 1, (4, 2))

# Train the model
for i in range(steps):
    gradients = qml.grad(cost, argnum=0)(weights, X_train_scaled, Y_train)

    # Update the weights
    optimizer.step(gradients, weights)

    # Print the cost
    if (i + 1) % 10 == 0:
        print(f'Step {i + 1}: cost = {cost(weights, X_train_scaled, Y_train):.4f}')

predictions = quantumVariationalClassifier(weights, X_test_scaled)

accuracy = qml.accuracy(predictions, y_test)

print(f'Test accuracy: {accuracy:.2f}')

plt.scatter(y_test, predictions)

# Add a diagonal line
x = np.linspace(0, 3, 4)
plt.plot(x, x, '--r')

# Add axis labels and a title
plt.xlabel('True labels')
plt.ylabel('Predictions')
plt.title('Quantum Neural Network')

# Show the plot
plt.show()





TypeError: QSharpCallable.__call__() takes 1 positional argument but 4 were given